In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('sample-orders.csv',encoding = "ISO-8859-1")

In [3]:
df.head()

,order_date,order_id,customer,grand_total
0,9/7/11,CA-2011-100006,Dennis Kane,378
1,7/8/11,CA-2011-100090,Ed Braxton,699
2,3/14/11,CA-2011-100293,Neil Franzsisch,91
3,1/29/11,CA-2011-100328,Jasper Cacioppo,4
4,4/8/11,CA-2011-100363,Jim Mitchum,21


In [4]:
df.columns

Index(['order_date', 'order_id', 'customer', 'grand_total'], dtype='object')

In [5]:
df.order_date.dtype

dtype('O')

In [6]:
max(df.order_date.values)

'9/9/14'

## The latest date found is  '9/9/14' 

### Suppose we are analysing data on end of 2014 ie 2014,12,31

In [7]:
import datetime
now = datetime.datetime(2014,12,31)

## Convert the order_date column of dataframe to datetime format

In [8]:

df['order_date'] = pd.to_datetime(df['order_date'])

In [9]:
df.head()

,order_date,order_id,customer,grand_total
0,2011-09-07,CA-2011-100006,Dennis Kane,378
1,2011-07-08,CA-2011-100090,Ed Braxton,699
2,2011-03-14,CA-2011-100293,Neil Franzsisch,91
3,2011-01-29,CA-2011-100328,Jasper Cacioppo,4
4,2011-04-08,CA-2011-100363,Jim Mitchum,21


# Create A RFM table:

## Tools for RFM (recency, frequency and monetary) analysis.

Introduction

    RFM (recency, frequency, monetary) analysis is a behavior based technique used to segment customers by examining their transaction history such as

        1. how recently a customer has purchased (recency)
        2. how often they purchase (frequency)
        3. how much the customer spends (monetary)

    It is based on the marketing axiom that 80% of your business comes from 20% of your customers. RFM helps to identify customers who are more likely to respond to promotions by segmenting them into various categories
    
 
 
Data:

   To calculate the RFM score for each customer we need transaction data which should include the following:

    1. a unique customer id
    2. date of transaction/order
    3. transaction/order amount



In [10]:
rfmTable = df.groupby('customer').agg({'order_date': lambda x: (now - x.max()).days, # Recency
                                        'order_id': lambda x: len(x),      # Frequency
                                        'grand_total': lambda x: x.sum()}) # Monetary Value

In [11]:
rfmTable.head()

,order_date,order_id,grand_total
customer,,,
Aaron Bergman,415,3,887
Aaron Hawkins,12,7,1744
Aaron Smayling,88,7,3050
Adam Bellavance,54,8,7756
Adam Hart,34,10,3249


In [12]:
rfmTable.order_date.dtype

dtype('int64')

Lets Change Column name to:

    Name               To
    order_date         recency
    order_id           frequency
    grand_total        monetery value
    

In [13]:
rfmTable.rename(columns={'order_date': 'recency', 
                         'order_id': 'frequency', 
                         'grand_total': 'monetary_value'}, inplace=True)

In [14]:
rfmTable.head(5)

,recency,frequency,monetary_value
customer,,,
Aaron Bergman,415,3,887
Aaron Hawkins,12,7,1744
Aaron Smayling,88,7,3050
Adam Bellavance,54,8,7756
Adam Hart,34,10,3249


In [15]:
df.loc[df.customer=='Aaron Bergman']

,order_date,order_id,customer,grand_total
624,2011-02-19,CA-2011-152905,Aaron Bergman,13
665,2011-03-07,CA-2011-156587,Aaron Bergman,310
2336,2013-11-11,CA-2013-140935,Aaron Bergman,564


In [16]:
import matplotlib.pyplot as plt

Customer 'Aaron Bergman' appeared 3 times in shop with total = 887 monetary_value

### Determining RFM Quartiles

    pandas.DataFrame.quantile

        DataFrame.quantile(self, q=0.5, axis=0, numeric_only=True, interpolation='linear')

    Return values at the given quantile over requested axis.

In [17]:
quantiles = rfmTable.quantile(q=[0.20,0.40,0.60,0.80])

In [18]:
quantiles

,recency,frequency,monetary_value
0.2,26.0,4.0,961.4
0.4,53.0,5.0,1739.8
0.6,103.0,7.0,2750.8
0.8,224.6,8.6,4282.6


In [19]:
quantiles = quantiles.to_dict()
quantiles

{'recency': {0.2: 26.0, 0.4: 53.0, 0.6: 103.0, 0.8: 224.60000000000002},
 'frequency': {0.2: 4.0, 0.4: 5.0, 0.6: 7.0, 0.8: 8.600000000000023},
 'monetary_value': {0.2: 961.4, 0.4: 1739.8, 0.6: 2750.8, 0.8: 4282.6}}

### Create Segmentation table for RMF

    As an example, suppose you have transaction data that looks like the following. You need only the 3 fields — customer    ID, a transaction date, and a transaction amount (value)
    
    
    From this, RFM calculates “scores” for each customer. The customers with the highest scores will probably be those that spent the most with you, across the most recent and frequent dates. The idea behind RFM is that a minority of customers are responsible for a majority of your business. RFM scores provide visibility into who those valuable customers are.

### RFM segmentation readily answers these questions for your business…

    1. Who are my best customers?
    2. Which customers are at the verge of churning?
    3. Who has the potential to be converted in more profitable customers?
    4. Who are lost customers that you don’t need to pay much attention to?
    5. Which customers you must retain?
    6. Who are your loyal customers?
    7. Which group of customers is most likely to respond to your current campaign?
    
First of all, it’s based on the Pareto Principle – commonly referred to as the 80-20 rule.
    Pareto’s rule says 80% of the results come from 20% of the causes.

    Similarly, 20% customers contribute to 80% of your total revenue.

    People who spent once are more likely to spend again. People who make big ticket purchases are more likely to repeat them.

    Pareto Principle is at the core of RFM model. Focusing your efforts on critical segments of customers is likely to give you much higher return on investment!
    
   

# RFM Score Calculations Simplified

## Wondering how to calculate RFM scores for your customer database? Here’s how…

### We need a few details of each customer:

    1. Customer ID / Email / Name etc – to identify them
    2. Recency (R) as days since last purchase: How many days ago was their last purchase? Deduct most recent purchase date from today to calculate the recency value. 1 day ago? 14 days ago? 500 days ago?
    3. Frequency (F) as total number of transactions: How many times has the customer purchased from our store? For example, if someone placed 10 orders over a period of time, their frequency is 10.
    4. Monetary (M) as total money spent: How many $$ (or whatever is your currency of calculation) has this customer spent? Again limit to last two years – or take all time. Simply total up the money from all transactions to get the M value.
    
    
 Link :   https://www.putler.com/rfm-analysis/
 
## The lowest recency, highest frequency and monetary amounts are our best customers.

In [20]:
# Arguments (x = value, p = recency / monetary_value / frequency, d = quartiles dict)
def RClass(x,p,d):
#     print(x) # value passed in each iteration
#     print(p) # recency
#     print(d) # dictionary of quantiles
    if x <= d[p][0.20]:
        return 5
    elif x <= d[p][0.40]:
        return 4
    elif x <= d[p][0.60]: 
        return 3
    elif x <= d[p][0.80]: 
        return 2
    else:
        return 1
    
# Arguments (x = value, p = recency / monetary_value / frequency, d = quartiles dict)
def FMClass(x,p,d):
    if x <= d[p][0.20]:
        return 1
    elif x <= d[p][0.40]:
        return 2
    elif x <= d[p][0.60]: 
        return 3
    elif x <= d[p][0.80]: 
        return 4
    else:
        return 5

In [21]:
rfmSegmentation = rfmTable
rfmSegmentation.head(2)

,recency,frequency,monetary_value
customer,,,
Aaron Bergman,415,3,887
Aaron Hawkins,12,7,1744


In [22]:
rfmSegmentation['R_Quartile'] = rfmSegmentation['recency'].apply(RClass, args=('recency',quantiles))

In [23]:
rfmSegmentation['F_Quartile'] = rfmSegmentation['frequency'].apply(FMClass, args=('frequency',quantiles))

In [24]:
rfmSegmentation['M_Quartile'] = rfmSegmentation['monetary_value'].apply(FMClass, args=('monetary_value',quantiles))

In [25]:
rfmSegmentation.head(4)

,recency,frequency,monetary_value,R_Quartile,F_Quartile,M_Quartile
customer,,,,,,
Aaron Bergman,415,3,887,1,1,1
Aaron Hawkins,12,7,1744,5,3,3
Aaron Smayling,88,7,3050,3,3,4
Adam Bellavance,54,8,7756,3,4,5


# OR

In [26]:
# r_labels = range(5, 1, -1)
# r_quartiles = pd.qcut(rfmSegmentation['recency'], 4, labels = r_labels)
# rfmSegmentation['R_Quartile'] = r_quartiles.values
# # Frequency Labels
# f_labels = range(1,5)
# f_quartiles = pd.qcut(rfmSegmentation['frequency'], 4, labels = f_labels)
# rfmSegmentation['F_Quartile'] = f_quartiles.values
# # Monetary Labels
# m_labels = range(1,5)
# m_quartiles = pd.qcut(rfmSegmentation['monetary_value'], 4, labels = m_labels)
# rfmSegmentation['M_Quartile'] = m_quartiles.values


> How ever this approach data at series are in form of categories so extra effort is needed 
to convert them to string type so we wont use this method here. However, this categores type
can be used in KMeans Clustering algorithms .

In [27]:
rfmSegmentation.head()

,recency,frequency,monetary_value,R_Quartile,F_Quartile,M_Quartile
customer,,,,,,
Aaron Bergman,415,3,887,1,1,1
Aaron Hawkins,12,7,1744,5,3,3
Aaron Smayling,88,7,3050,3,3,4
Adam Bellavance,54,8,7756,3,4,5
Adam Hart,34,10,3249,4,5,4


In [28]:
rfmSegmentation.R_Quartile

customer
Aaron Bergman         1
Aaron Hawkins         5
Aaron Smayling        3
Adam Bellavance       3
Adam Hart             4
                     ..
Xylona Preis          4
Yana Sorensen         5
Yoseph Carroll        5
Zuschuss Carroll      3
Zuschuss Donatelli    2
Name: R_Quartile, Length: 793, dtype: int64

In [29]:
#

In [30]:
 rfmSegmentation.R_Quartile.map(str) + rfmSegmentation.F_Quartile.map(str)

customer
Aaron Bergman         11
Aaron Hawkins         53
Aaron Smayling        33
Adam Bellavance       34
Adam Hart             45
                      ..
Xylona Preis          45
Yana Sorensen         54
Yoseph Carroll        52
Zuschuss Carroll      35
Zuschuss Donatelli    22
Length: 793, dtype: object

## Add a new column to combine RFM score: 

In [31]:
rfmSegmentation['RFMClass'] = rfmSegmentation.R_Quartile.map(str)+ rfmSegmentation.F_Quartile.map(str)+ rfmSegmentation.M_Quartile.map(str)

In [32]:
rfmSegmentation.head()

,recency,frequency,monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMClass
customer,,,,,,,
Aaron Bergman,415,3,887,1,1,1,111
Aaron Hawkins,12,7,1744,5,3,3,533
Aaron Smayling,88,7,3050,3,3,4,334
Adam Bellavance,54,8,7756,3,4,5,345
Adam Hart,34,10,3249,4,5,4,454




## Who are the top 5 best customers? by RFM Class (555), high spenders who buy recently and frequently?


In [33]:
rfmSegmentation[rfmSegmentation['RFMClass']=='555'].sort_values('monetary_value', ascending=False).head(5)

,recency,frequency,monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMClass
customer,,,,,,,
Sanjit Engle,9,11,12210,5,5,5,555
John Lee,21,11,9801,5,5,5,555
Pete Kriz,9,12,8647,5,5,5,555
Harry Marie,2,10,8237,5,5,5,555
Lena Creighton,16,12,7661,5,5,5,555
